In [ ]:
"""

Prepare clean stereographic grid for Justine's file with bounds and so on

"""

In [8]:
import xarray as xr
from cdo import Cdo
import multimelt.useful_functions as uf
from tqdm.notebook import tqdm
import numpy as np

In [17]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'

In [6]:
ISMIP_2km_grid = xr.open_dataset(inputpath_raw + 'Mask_Iceshelf_IMBIE2_v2.nc')

In [9]:
meshx,meshy = np.meshgrid(ISMIP_2km_grid.x,ISMIP_2km_grid.y)

In [10]:
half_res = 1000.

In [11]:
meshlon_ll,meshlat_ll = uf.change_coord_stereo_to_latlon(meshx - half_res, meshy - half_res)
meshlon_lr,meshlat_lr = uf.change_coord_stereo_to_latlon(meshx + half_res, meshy - half_res)
meshlon_ur,meshlat_ur = uf.change_coord_stereo_to_latlon(meshx + half_res, meshy + half_res)
meshlon_ul,meshlat_ul = uf.change_coord_stereo_to_latlon(meshx - half_res, meshy + half_res)

In [12]:
meshlon,meshlat = uf.change_coord_stereo_to_latlon(meshx, meshy)

In [13]:
lon_bnds = xr.DataArray(data=np.array([meshlon_ll, meshlon_lr, meshlon_ur, meshlon_ul]), dims=['nv4','y','x'])
lat_bnds = xr.DataArray(data=np.array([meshlat_ll, meshlat_lr, meshlat_ur, meshlat_ul]), dims=['nv4','y','x'])

In [14]:
lon = xr.DataArray(data=np.array(meshlon), dims=['y','x'])
lat = xr.DataArray(data=np.array(meshlat), dims=['y','x'])

In [15]:
ISMIP_2km_new_grid = xr.merge([lon_bnds.rename('lon_bnds'),lat_bnds.rename('lat_bnds')]).assign_coords({'lon': lon, 'lat': lat, 'y': ISMIP_2km_grid.y, 'x': ISMIP_2km_grid.x})

In [19]:
ISMIP_2km_new_grid.transpose('y','x','nv4').to_netcdf(inputpath_interim + 'IMBIE_2km_stereo_withbnds.nc')

In [20]:
ISMIP_2km_new_grid[['lon','lat']].to_netcdf(inputpath_interim + 'IMBIE_2km_stereo_mapping.nc')

In [25]:
3041 * 3041

9247681

In [27]:
with open(inputpath_interim+"stereo_grid_2km_curvilinear_definition.txt", "w") as txt_file:

    txt_file.write('# \n')
    txt_file.write('# gridID 1 \n')
    txt_file.write('# \n')
    txt_file.write('gridtype  = curvilinear \n')
    txt_file.write('gridsize  = 9247681 \n')
    txt_file.write('datatype  = float \n')
    txt_file.write('xsize     = 3041 \n')
    txt_file.write('ysize     = 3041 \n')
    txt_file.write('xname     = lon \n')
    txt_file.write('xdimname  = x \n')
    txt_file.write('xlongname = "longitude" \n')
    txt_file.write('xunits    = "degrees_east" \n')
    txt_file.write('yname     = lat \n')
    txt_file.write('ydimname  = y \n')
    txt_file.write('ylongname = "latitude" \n')
    txt_file.write('yunits    = "degrees_north" \n')
    txt_file.write('vdimname  = nv4 \n')
    txt_file.write('xvals = ')
    for vv in tqdm(meshlon.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('yvals = ')
    for vv in tqdm(meshlat.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('xbounds = ')
    for vv in tqdm(ISMIP_2km_new_grid['lon_bnds'].values.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('ybounds = ')
    for vv in tqdm(ISMIP_2km_new_grid['lat_bnds'].values.flatten()):
        txt_file.write(str(vv) + ' ')
    txt_file.write('\n')
    txt_file.write('gridtype = projection \n')
    txt_file.write('gridsize  = 9247681 \n')
    txt_file.write('xsize     = 3041 \n')
    txt_file.write('ysize     = 3041 \n')
    txt_file.write('xname     = x \n')
    txt_file.write('xlongname = "x-coordinate in Cartesian system" \n')
    txt_file.write('xunits = "m" \n')
    txt_file.write('yname     = y \n')
    txt_file.write('ylongname = "y-coordinate in Cartesian system" \n')
    txt_file.write('yunits = "m" \n')
    txt_file.write('xfirst = '+str(ISMIP_2km_new_grid.x.min().values)+' \n')
    txt_file.write('xinc = 2000 \n')
    txt_file.write('yfirst = '+str(ISMIP_2km_new_grid.y.min().values)+' \n')
    txt_file.write('yinc = 2000 \n')
    txt_file.write('grid_mapping = Projection \n')
    
txt_file.close()

  0%|          | 0/9247681 [00:00<?, ?it/s]

  0%|          | 0/9247681 [00:00<?, ?it/s]

  0%|          | 0/36990724 [00:00<?, ?it/s]

  0%|          | 0/36990724 [00:00<?, ?it/s]